# Convert Strawberry Data to MTG

**Authors** : Marc Labadie, Christophe Pradal

In [1]:
from openalea.deploy.shared_data import shared_data
import strawberry
from strawberry.reader import Reader, strawberry2mtg 
from openalea.mtg import MTG, fat_mtg
from openalea.mtg.traversal import *
from math import radians

ImportError: No module named strawberry

In [2]:
from openalea.deploy.shared_data import shared_data
import strawberry


ImportError: No module named strawberry

In [6]:
files = shared_data(strawberry).glob('*.csv')
fn = files[13]
print fn

NameError: name 'strawberry' is not defined

Reading the file and create an MTG

In [7]:
gs = [strawberry2mtg(fn) for fn in files]
g = gs[13]

NameError: name 'files' is not defined

## Method that Computes the Phyllochron

Parameters : MTG and the delay between the appartion of two successive organs on the same axis
This delay can be express in degree day.
The result is : a MTG property named age that is retrieve to show the development of the plant through time.
The hypothesis is that the delay is constant and the time of apparition of a lateral organ is 10 times the delay.

In [8]:
def compute_pyllochrone(g, dt = 10.):
    max_scale = g.max_scale()
    age = {}
    for v in pre_order(g, 1):
        pid = g.parent(v)
        if pid is None:
            age[v] = 0.
        else:
            age[v] = age[pid]+dt
            if g.edge_type(v) == '+':
                age[v] += 10*dt
    g.properties()['age'] = age
    return g

In [9]:
from openalea.mtg import traversal
from openalea.mtg.plantframe import turtle
from openalea.plantgl.all import *
%gui qt

g = gs[8]

NameError: name 'gs' is not defined

In [10]:
# compute orders
from openalea.mtg.algo import orders

#add a order property order
g.properties()['order'] = orders(g)


# phyllotaxis
roll_angle = {}

ramifs = [v for v in g.vertices(scale=g.max_scale()) if g.edge_type(v)!='<']
for axis_id in ramifs:
    for i, v in enumerate(g.Axis(axis_id)):
        if i%2:
            roll_angle[v] = 180.
        elif i == 2:
            roll_angle[v] = 90.
        else:
            roll_angle[v] = 45
g.properties()['roll_angle'] = roll_angle

NameError: name 'g' is not defined

In [11]:
def leaflet(length=1., width=1.):
    disc = Translated((-0.5,0,0), Disc())
    disc = Scaled((length, width,1), disc)
    disc = AxisRotated(axis=(0,1,0), angle=radians(90.), geometry=disc)
    
    d1 = AxisRotated(axis=(1,0,0), angle=-radians(60.), geometry=disc)
    d2 = AxisRotated(axis=(1,0,0), angle=-radians(-60.), geometry=disc)
    d3 = AxisRotated(axis=(1,0,0), angle=0., geometry=disc)
    
    shape = Group([d1, d2, d3])
    return shape

# Compute Phyllotaxy
def Phytomer(g, vid, turtle):
    t = turtle
    nid = g.node(vid)
    order = nid.order
    t.setColor(2+order)
    t.setWidth(0.01)
    # set color
    
    len_petiole = 0.7
    len_internode = 0.1
    leaflet_length = 0.7/2.
    leaflet_wdth = 0.3/2.
    
    t.F(0.1)
    #if order != 1:
    #    return
    t.push()
    t.down(45.)
    t.F(len_petiole)
    t.customGeometry(leaflet(leaflet_length, leaflet_wdth))
    t.pop()

def TerminalBud(g, vid, turtle):
    t = turtle
    #turtle.setColor(1)
    nid = g.node(vid)
    order = nid.order
    t.setColor(2+order)
    turtle.F(0.2)
    sphere = Sphere(radius=0.05)
    turtle.customGeometry(sphere)    

def Inflorescence(g, vid, turtle):
    """
    """
    t = turtle
    nid = g.node(vid)
    order = nid.order
    #turtle.setColor(3)
    t.setColor(2+order)
    turtle.F(0.2)
    sphere = Box(0.05*Vector3(1,1,1))
    turtle.customGeometry(sphere)

In [12]:
count = 0
roll_angle = g.property('roll_angle')
def strawberry_visitor(g, v, turtle, time=0):
    """ Function that draw geometry for a given vertex. """
    global count
    count+=1
    turtle.setWidth(0.01)
    nid = g.node(v)

    if v in roll_angle:
        turtle.rollL(nid.roll_angle)
    else:
        turtle.rollL()
   
    #if nid.order != 1:
    #    return
    
    if g.edge_type(v) == '+':
        turtle.down(30)
    label = g.label(v)
    #print label
    turtle.setId(v)
    if label == 'Phytomer':
        Phytomer(g, v, turtle)
    elif label == 'TerminalBud':
        TerminalBud(g, v, turtle)
    elif label == 'Inflorescence':
        Inflorescence(g, v, turtle)

scene = turtle.traverse_with_turtle(g, 1, visitor=strawberry_visitor)
Viewer.display(scene)
print count

NameError: name 'g' is not defined

In [13]:
def mtg_turtle_time(g, time, visitor=strawberry_visitor, time_property='age' ):
    ''' Compute the geometry on each node of the MTG using Turtle geometry. 
    
    Update_visitor is a function called on each node in a pre order (parent before children).
    This function allow to update the parameters and state variables of the vertices.
    
    :Example:

        >>> def grow(node, time):

    '''

    g.properties()['geometry'] = {}
    orders = g.property('order')
    max_scale = g.max_scale()

    start_dates = g.property(time_property)
    

    def traverse_with_turtle_time(g, vid, time, visitor=visitor):
        turtle = PglTurtle()
        def push_turtle(v):
            try:
                start_tt = start_dates.get(v, time+1)
                if start_tt > time:
                    return False
            except: 
                pass
            if g.edge_type(v) == '+':
                turtle.push()
            return True

        def pop_turtle(v):
            try:
                start_tt = start_dates.get(v, time+1)
                if start_tt > time:
                    return False
            except: 
                pass
            if g.edge_type(v) == '+':
                turtle.pop()

        if start_dates[vid] <= time:
            visitor(g,vid,turtle,time)
            #turtle.push()
        
        for v in pre_order2_with_filter(g, vid, None, push_turtle, pop_turtle):
            if v == vid: 
                continue
            # Done for the leaves
            if start_dates.get(v,time+1) > time:
                print 'Do not consider ', v, time
                continue
            visitor(g,v,turtle,time)

        scene = turtle.getScene()
        return scene, g

    for plant_id in g.component_roots_at_scale_iter(g.root, scale=max_scale):
        scene, g = traverse_with_turtle_time(g, plant_id, time)
    return g, scene

NameError: name 'strawberry_visitor' is not defined

In [ ]:
compute_pyllochrone(g)

In [ ]:
max(g.property('age').values())

In [ ]:
for t in range(1,790):
    g, scene = mtg_turtle_time(g, t)
    Viewer.display(scene)